In [0]:
#Mount drive files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install --upgrade tensorflow==1.13.1
!pip install --upgrade pandas
!pip install pyunpack
!pip install patool

#!pip install -q tensorflow-gpu==2.0.0-alpha0
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import json
from pyunpack import Archive
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [0]:
ab_filepath = 'drive/My Drive/fyp/bert_abstract_all.7z'

abstract_path = 'bert_abstract_all.json'
max_seq_len = 6
embedd_len = 768

In [0]:
import glob, os
m_files_dir = 'drive/My Drive/fyp/bert_maintext_csv'
os.chdir(m_files_dir)
for file in glob.glob("*.csv"):
  print(file)

seg_1.csv
seg_2.csv
seg_3.csv
seg_4.csv
seg_5.csv
seg_6.csv


In [0]:
import glob, os
m_files_dir = 'drive/My Drive/fyp/bert_maintext_csv'
max_maintext_seq_len = 100
arr_type = np.dtype(list)
segments = []
for file in glob.glob("*.csv"):
    seg_df = pd.read_csv(file, names=['label', 'embedding'], dtype={1:arr_type}, header=None, sep='\t')
    segments.append(seg_df)
    print(file, 'loaded')

df = pd.concat(segments, axis=0, ignore_index=True)
    

seg_1.csv loaded
seg_2.csv loaded
seg_3.csv loaded
seg_4.csv loaded
seg_5.csv loaded
seg_6.csv loaded


In [0]:
type(df['embedding'][0])

str

In [0]:
def str_to_list(x):
  if(isinstance(x, str)):
    return ast.literal_eval(x)
  else:
    return x

In [0]:
import ast
total = len(df['embedding'])
for index, x in enumerate(df['embedding']):
  df['embedding'][index] = str_to_list(df['embedding'][index])
  df['embedding'][index] = pad_paper(df['embedding'][index], 100, 768)
  df['embedding'][index] = np.array(df['embedding'][index])
  
  if(index % 100 == 0):
    print('progress:',index,'/',total,'\tshape:',df['embedding'][index].shape)

progress: 0 / 5488 	shape: (100, 768)
progress: 100 / 5488 	shape: (100, 768)
progress: 200 / 5488 	shape: (100, 768)
progress: 300 / 5488 	shape: (100, 768)
progress: 400 / 5488 	shape: (100, 768)
progress: 500 / 5488 	shape: (100, 768)
progress: 600 / 5488 	shape: (100, 768)
progress: 700 / 5488 	shape: (100, 768)
progress: 800 / 5488 	shape: (100, 768)
progress: 900 / 5488 	shape: (100, 768)
progress: 1000 / 5488 	shape: (100, 768)
progress: 1100 / 5488 	shape: (100, 768)
progress: 1200 / 5488 	shape: (100, 768)
progress: 1300 / 5488 	shape: (100, 768)
progress: 1400 / 5488 	shape: (100, 768)
progress: 1500 / 5488 	shape: (100, 768)
progress: 1600 / 5488 	shape: (100, 768)
progress: 1700 / 5488 	shape: (100, 768)
progress: 1800 / 5488 	shape: (100, 768)
progress: 1900 / 5488 	shape: (100, 768)
progress: 2000 / 5488 	shape: (100, 768)
progress: 2100 / 5488 	shape: (100, 768)
progress: 2200 / 5488 	shape: (100, 768)
progress: 2300 / 5488 	shape: (100, 768)
progress: 2400 / 5488 	shape

In [0]:
for index, x in enumerate(df['embedding']):
  if(isinstance(x, str)):
    print(index)
    break

1271


In [0]:
import gc
segments = 0
gc.collect()

255

pandas.core.series.Series

In [0]:

def pad_paper(paper, max_seq_length, vec_length):
  padding = vec_length * [0]
  paper_len = len(paper)
  outpaper = paper
  
  if (paper_len > max_seq_length):
    outpaper = paper[:max_seq_length]
    
  elif(paper_len < max_seq_length):
    pad_no = max_seq_length - paper_len
    
    for pad in range(pad_no):
      outpaper.append(padding) 
    
  else:
    outpaper = paper
  return outpaper

In [0]:
##Archive(ab_filepath).extractall('')

##with open(abstract_path) as json_file: 
  ##abstracts = json.load(json_file)
  ##print(len(abstracts))

In [0]:
# new_abs = []

# size = len(abstracts)
# for ab in abstracts:
#     vectors = ab['abstract_sentence_all']
#     new_vectors = pad_paper(vectors, 10, 768)
#     new_ab = ab 
#     new_ab['abstract_sentence_all'] = new_vectors
#     new_abs.append(new_ab)



In [0]:
size = len(df)
for index, x in enumerate(df['embedding']):
    vec = df['embedding'][index]
    df['embedding'][index] = pad_paper(vec, 100, 768)
    
    if(index % 100 == 0):
      print('progress:\t',index,'/',total)
    

In [0]:
##abstracts = new_abs

In [0]:
#for ab in abstracts[0]['abstract_sentence_all']:
##  print(ab)#

In [0]:
# df = pd.DataFrame(abstracts)
# feature = 'abstract_sentence_all'

In [0]:
df.head()

,label,embedding
0,evolutionary biology,"[[0.7809642553329468, 0.24800170958042145, -0...."
1,molecular biology,"[[0.2271534502506256, -0.4300555884838104, -0...."
2,molecular biology,"[[0.9033320546150208, 0.02782544679939747, -0...."
3,genetics,"[[0.8269822597503662, 0.1830005943775177, -0.0..."
4,bioinformatics,"[[0.7169336080551147, 0.1673220545053482, -0.2..."


In [0]:
for index,emb in enumerate(df.embedding):
  if emb.shape != (100,768):
    print(index)
    break

In [0]:
feature = 'embedding'
X = np.array([np.array(a) for a in df[feature].values])
Y = df.label
classes_size = Y.value_counts().shape[0]
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [0]:
print(X.shape)
print(Y.shape)

(5488, 100, 768)
(5488, 1)


In [0]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [0]:
X_train.shape

(4664, 100, 768)

In [0]:
type(X_train[0][0])

In [0]:
classes_size

In [0]:
max_seq_len = 100
embedd_len = 768
X_sequence = X_train

In [0]:
from keras.utils import to_categorical
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

Using TensorFlow backend.


In [0]:

inputs = tf.keras.layers.Input(name='inputs',shape=[max_seq_len,embedd_len])
layer = tf.keras.layers.LSTM(64)(inputs)
layer = tf.keras.layers.Dense(256,name='FC1')(layer)
layer = tf.keras.layers.Activation('relu')(layer)
layer = tf.keras.layers.Dropout(0.5)(layer)
#layer = tf.keras.layers.Dense(64, activation='relu')(layer)
layer = tf.keras.layers.Dense(classes_size,name='out_layer')(layer)
layer = tf.keras.layers.Activation('softmax')(layer)
model = tf.keras.models.Model(inputs=inputs,outputs=layer)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 100, 768)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                213248    
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 27)                6939      
_________________________________________________________________
activation_1 (Activation)    (None, 27)                0         
Total para

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

In [0]:
history = model.fit(X_train,Y_train,batch_size=64,epochs=2,
          validation_split=0.2, callbacks = [cp_callback])

Train on 3731 samples, validate on 933 samples
Epoch 1/2
3712/3731 [============================>.] - ETA: 0s - loss: 1.6484 - acc: 0.5022
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3731/3731 [==============================] - 13s 3ms/sample - loss: 1.6482 - acc: 0.5031 - val_loss: 1.7373 - val_acc: 0.4909
Epoch 2/2
3712/3731 [============================>.] - ETA: 0s - loss: 1.5970 - acc: 0.5097
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
3731/3731 [==============================] - 12s 3ms/sample - loss: 1.5955 - acc: 0.5098 - val_loss: 1.6545 - val_acc: 0.5166


In [0]:
model.save('bert_maintext_all.h5')

In [0]:
accr = model.evaluate(X_test,Y_test)

824/824 [==============================] - 2s 2ms/sample - loss: 1.6777 - acc: 0.5085


In [0]:
test = model.predict(X_test[:1])

In [0]:
X_test[:1].shape

In [0]:
X_test[:1].shape